## Monte Carlo simulation for NVT Ensemble

Reference: Computer Simulation of Liquid, 2017

Import packages

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from tqdm import tqdm
from itertools import product

All particles are set at the center of a simple square lattice, at the initial state

In [17]:
def initCoords(pos, cells):
    num = len(pos)
    dim = len(pos[0])
    c   = np.zeros(dim)
    gap = 1. / cells

    assert dim == 3, "Wrong dimension, initCoords"

    n = 0
    for i, j, k in product(range(cells[0]), range(cells[1]), range(cells[2]) ):

        c      = np.array([i, j, k]) + 0.5
        c      = c * gap
        c      = c - 0.5
        pos[n] = c
        n      = n + 1

        if n >= num:
            return pos

    return pos


In [20]:
class PotentialType:
    """A compsite variable for interactions."""

    def __init__(self, pot, vir, ovr):
        self.pot = pot # Potential Energy
        self.vir = vir # Virial
        self.ovr = ovr # Check if overlap

    def __add__(self, other):
        pot = self.pot + other.pot
        vir = self.vir + other.vir
        ovr = self.ovr or other.ovr
        return PotentialType(pot, vir, ovr)

    def __sub__(self, other):
        pot = self.pot - other.pot
        vir = self.vir - other.vir
        ovr = self.ovr or other.ovr
        return PotentialType(pot, vir, ovr)


Calculate the interaction between particle $i$ and other particles.
The Lennard-Jones potential is used.

In [ ]:
def potential_1( ri, box, r_cut, r ):

    dim          = len(r[0])
    r_cut_box    = r_cut / box
    r_cut_box_sq = r_cut_box ** 2
    box2         = box ** 2

    assert dim == 3, "Dimension error for r in potential_1"

    sr2_ovr = 1.77 # Overlap threshold (pot > 100)

    rij = ri - r
    rij = rij - np.rint(rij)  # Periodical boundary conditions
    rij2 = np.sum(rij**2, axis = 1)
    rij2 = rij2 * box2
    in_range = rij2 < r_cut_box_sq
    sr2 = np.where(in_range, 1./rij2, 0)

    ovr = sr2 > sr2_ovr
    if np.any(ovr):
        partial = PotentialType( pot = 0.0, vir = 0.0, ovr = True)
        return partial

    sr6 = sr2 ** 3
    sr12 = sr6 ** 2

    pot = sr12 - sr6
    vir = pot + sr12
    partial = PotentialType ( pot=np.sum(pot), vir=np.sum(vir), ovr=False)

    partial.pot = partial.pot * 4.0          # 4 * epsilon
    partial.vir = partial.vir * 24.0 / 3.0   # 24 * epsilon and divide by 3.0

    return partial

def potential( box, r_cut, r ):
    """
    Calculate the total energy of the system
    """

    dim = len(r[0])
    num = len(r)
    assert dim == 3, "Dimension error for r in potential"

    total = PotentialType ( pot=0.0, vir=0.0, ovr=False )

    for i in range(num-1):
        partial = potential_1 ( r[i,:], box, r_cut, r[i+1:,:] )
        if partial.ovr:
            total.ovr = True
            break
        total = total + partial

    return total

